In [3]:
from cProfile import label
import os
from pyclbr import Function
from turtle import color
from xml.etree.ElementTree import PI
import math 
from sympy import apart
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from dolfin import *
from matplotlib import pyplot
from ufl import nabla_div, nabla_grad,bessel_I,bessel_K
import numpy as np
import  sys
import mshr
from label_lines import *
import mpmath 

ModuleNotFoundError: No module named 'label_lines'

In [4]:
def besseli(n,f):
    return bessel_I(n, f).evaluate(1,1,1,1)
def besselk(n,f):
    if n==0:
        bes = bessel_K(n, f).evaluate(1,1,1,1)
    else:
        bes = mpmath.besselk(n,f)
    return bes

R_=300
H=10


# B_s=1E-11
# B_m=1E-10
B_f= 1E-9
Poro=0.4

K_m=500E3
nu=4/11#coeficiente de poisson  
E=3*K_m*(1-2*nu)#(9*K_m*G_m)/(3*K_m+G_m)# #modulo elasticidad
B_m=(K_m)**(-1)
B_s=0
alpha=1#/(1-B_s/B_m) #biotcoef
s_coef=0#(alpha-Poro)*B_s +Poro*B_f

theta =18.94#K(subd,18.94,20.61,23.27,20.53,21.84) #angulos friccion interna
C=15530#K(subd,15530,10350,18650,18400,14000) #cohesion

#E=B_m**(-1)*3*(1-2*nu)#modulo elasticidad 
print('modulo elasticidad ',E)
mu =E/(2*(1+nu))#coeficientes de Lame
print("relacion K/G= ",1/(B_m*mu))
lmbda =(3*K_m*nu)/(1+nu) #E*nu/((1+nu)*(1-2*nu))#coeficientes de Lame


K=1.15741E-9


cv_dot =(K*(1/B_m+mu/3))/(alpha**2+s_coef*(1/B_m+mu/3))
cv =(K*(1/B_m+4*mu/3))/(alpha**2+s_coef*(1/B_m+4*mu/3))

print('cv*=   ',cv_dot)
print('cv=   ',cv)
#material 
t=0 # tiempo inicial
Ti=0.15#tiempo total


dtdot=0.001
delta= dtdot/(cv/R_**2)
steps=int( 1/dtdot)
dt=Constant((delta))
#boundary conditions 
Caudal=1.1574E-3
flo=Constant((Caudal/(2*math.pi*H)))



snaps=80


Betta= alpha/(alpha**2 + s_coef*(1/B_m+mu/3))
q_ = Caudal/(2*math.pi*H*K)
M=10
r = np.linspace(0.01, R_, snaps)

modulo elasticidad  409090.90909090906
relacion K/G=  3.333333333333334
cv*=    0.0006365755000000001
cv=    0.000810187


In [5]:

def pbar(r,s,f_const):
    ro=r/R_
    sigmma = (s * R_ ** 2) / cv_dot
    I_cero_ro = besseli(0, ro * (sigmma) ** (1 / 2))
    K_cero_ro = besselk(0, ro * (sigmma) ** (1 / 2))
    I_cero = besseli(0, (sigmma) ** (1 / 2))
    K_cero = besselk(0, (sigmma) ** (1 / 2))
    term_1 = (Betta * mu * f_const / q_) * (1 - I_cero_ro / I_cero)
    term_2 = (1 / sigmma) *( K_cero_ro-(K_cero / I_cero) * I_cero_ro)
    v_ray = (term_1 - term_2 )
    return v_ray

def f_coef(r,s,f_const):
    ro=r/R_
    sigmma = (s * R_ ** 2)/cv_dot
    part_1 = (2*alpha/sigmma**2)*(1-1/(besseli(0,np.sqrt(sigmma))))
    part_2 = 3/(B_m*mu) + alpha*Betta*(1-(2*besseli(1,np.sqrt(sigmma)))/(np.sqrt(sigmma)*besseli(0,np.sqrt(sigmma))))
    return (part_1/part_2)*q_/mu
def rad_displacement(r,s,f_const):
    ro = r/R_
    sigmma=(s * R_ ** 2) / cv_dot
    I_uno_ro = besseli(1, ro * (sigmma) ** (1 / 2))
    K_uno_ro = besselk(1, ro * (sigmma) ** (1 / 2))
    I_cero = besseli(0, (sigmma) ** (1 / 2))
    K_cero = besselk(0, (sigmma) ** (1 / 2))
    part_1_1 = ro/4- I_uno_ro/(2*sigmma**(1/2)*I_cero)
    part_1_2 = ro/4*(1/(mu*B_m)+4/3)
    part_1 = (alpha*Betta*part_1_1-part_1_2)*(mu*f_const/q_)
    part_2 = (alpha/(2*sigmma**(3/2)))*(K_uno_ro - 1/(ro*sigmma ** (1/2)) + (K_cero/I_cero)*I_uno_ro)
    n_ray = part_1 + part_2
    return n_ray

def talbot(M, t,Fun,r=0,f_const=0):
    v=0
    for k in range(M - 1):
        if k == 0:
            delta =2 * M / 5
            gamma = np.exp(delta) / 2
        else:
            cot = 1 / np.tan(k * np.pi / M)
            delta = (2 * k * np.pi / 5) * (cot + 1j)
            gamma = (1 + 1j * (k * np.pi / M) *(1+ (cot) ** 2) - 1j* cot) * np.exp(delta)
        term_fun = delta / t
        termino = (gamma * Fun(r,term_fun,f_const))
        v += termino.real
    v_final=((2) / (5 * t)) * v
    return float(v_final)
    

def p_analitico(r,M,t):
    f_const = talbot(M, t, f_coef)
    for l in range(len(r)):   
        if l==0:
            v_final=np.array([[talbot(M, t, pbar,r=r[0],f_const=f_const)*R_**2/cv_dot,r[l]/R_]])
        else:
            v_final= np.append(v_final,np.array([[talbot(M, t, pbar,r=r[l],f_const=f_const)*R_**2/cv_dot,r[l]/R_]]),axis=0)
    return v_final


def u_analitico(r, M, t):
    f_const = talbot(M, t, f_coef)
    for l in range(len(r)):
        if l == 0:
            u_final= np.array([[-talbot(M, t, rad_displacement, r=r[0], f_const=f_const)*R_**2/cv_dot, r[l]/R_]])
        else:
            u_final = np.append(u_final,np.array([[-talbot(M, t, rad_displacement, r=r[l], f_const=f_const)*R_**2/cv_dot, r[l]/R_]]),axis=0)
    return u_final
def u_analiticoP(r, M, t):
    print("u analico init")
    f_const = talbot(M, t, f_coef)
    u_final=-talbot(M, t, rad_displacement, r=r, f_const=f_const)
    return u_final*R_**2/cv_dot

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jun  1 14:26:23 2021
@author: SebastianG
"""



t=delta
f = plt.figure()
f.set_figwidth(10)
f.set_figheight(10)
L2=[]
uplot=[]
for pot in range(steps):
    tdot= (cv/R_**2)*t
    
    uplot.append([u_analiticoP(R_,M,t),0,tdot])
    print('step', pot, 'of', steps,'time*:',tdot)

    t += delta

ig, ax = plt.subplots()
uplot=np.array(uplot)
line1, = ax.plot(uplot[:, 2], -uplot[:, 0], "--", color='black', label='Analítica')
line2,=ax.plot(uplot[:,2],-uplot[:,1], "-",color='red',label='FEM')
ax.legend(handler_map={line1: HandlerLine2D(numpoints=4)},loc= 'upper left')
ax.set_xscale('log')
plt.xlabel("$t*$ ",fontsize=20)
plt.ylabel("$u*_{z}$",fontsize=20)
plt.grid(True,color='k',which="both",alpha=0.3, linestyle='-', linewidth=0.5)

plt.close()


u analico init


NameError: name 'mpmath' is not defined

<Figure size 720x720 with 0 Axes>